*AT&T spam detector*

### *Imports & Installations*

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 36.7 MB/s 
     |████████████████████████████████| 182 kB 49.5 MB/s 


In [ ]:
!pip install -q -U "tensorflow-text==2.9.*"

     |████████████████████████████████| 4.6 MB 4.7 MB/s 


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM
import tensorflow_text

import pathlib 

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.offline as py
import plotly.tools as tls

color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

import os
import io
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
warnings.filterwarnings("ignore")

pio.templates.default = "plotly_dark"
matplotlib.style.use('dark_background')

In [ ]:
tf.__version__

'2.9.2'

### *EDA & Data cleaning*

In [ ]:
# Import dataset with Pandas 
dataset = pd.read_csv("https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv", error_bad_lines=False, encoding='iso-8859-1')
dataset.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
dataset = dataset.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
dataset.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
dataset["type"] = dataset["v1"].apply(lambda x: 1 if x=="spam" else 0)

In [ ]:
dataset.head()

,v1,v2,type
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
# fig = px.bar

In [ ]:
dataset["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [ ]:
dataset["v2"].value_counts()

Sorry, I'll call later                                                                                                                                                 30
I cant pick the phone right now. Pls send a message                                                                                                                    12
Ok...                                                                                                                                                                  10
7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st \Ur Lovely Friendship\"... good morning dear"               4
Say this slowly.? GOD,I LOVE YOU &amp; I NEED YOU,CLEAN MY HEART WITH YOUR BLOOD.Send this to Ten special people &amp; u c miracle tomorrow, do it,pls,pls do it...     4
                                                                                                                                                      

*We notice that there is a lot of data with numbers and punctuations.*

### *Preprocessing*

In [ ]:
!python -m spacy download en_core_web_md -q

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-02 15:20:29.623994: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     |████████████████████████████████| 42.8 MB 4.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import en_core_web_md
nlp = en_core_web_md.load()

In [ ]:
# Import Stop words 
from spacy.lang.en.stop_words import STOP_WORDS

*We will now have to clean our texts in order to prepare them for training.*
*Let's do this in three different steps :*
 - using the command `str.isalnum` remove all characters from your strings that are not alphanumeric except for whitespaces.
 - using `str.replace`, `str.lower` and `str.strip` replace double whitespaces with single whitespaces, convert all characters to lowercase and trim starting and finishing whitespaces.
 - using spacy, replace all tokens in your texts with `lemma_` and remove all the stop words.

In [ ]:
dataset["msg_clean"] = dataset["v2"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" " or ch=="'"))
dataset["msg_clean"] = dataset["msg_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
dataset["msg_clean"] = dataset["msg_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))
dataset.head()

,v1,v2,type,msg_clean
0,ham,"Go until jurong point, crazy.. Available only ...",0,jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,0,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,nah think usf live


In [ ]:
mask = dataset.msg_clean.apply(lambda x: type(x)==str)
mask.value_counts()

True    5572
Name: msg_clean, dtype: int64

*Let's instanciate the tokenizer, we set it up to keep only the most 1000 common words*

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab") # instanciate the tokenizer
tokenizer.fit_on_texts(dataset.msg_clean)
dataset["msg_encoded"] = tokenizer.texts_to_sequences(dataset.msg_clean)

In [ ]:
dataset.head()

,v1,v2,type,msg_clean,msg_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",0,jurong point crazy available bugis n great wor...,"[1, 230, 445, 462, 941, 32, 49, 204, 942, 77, ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar joke wif u oni,"[8, 194, 463, 290, 2, 1]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win fa cup final tkts 2...,"[11, 302, 3, 531, 659, 30, 1, 845, 421, 1, 1, ..."
3,ham,U dun say so early hor... U c already then say...,0,u dun early hor u c,"[2, 125, 150, 1, 2, 79]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,nah think usf live,"[701, 20, 660, 131]"


In [ ]:
tokenizer.index_word

{1: 'out_of_vocab',
 2: 'u',
 3: '2',
 4: 'ur',
 5: 'come',
 6: '4',
 7: 'know',
 8: 'ok',
 9: 'ltgt',
 10: 'good',
 11: 'free',
 12: 'send',
 13: 'like',
 14: 'want',
 15: 'day',
 16: 'time',
 17: 'love',
 18: 'text',
 19: 'tell',
 20: 'think',
 21: 'need',
 22: 'today',
 23: 'txt',
 24: 'home',
 25: 'lor',
 26: 'r',
 27: 'reply',
 28: 'stop',
 29: 'sorry',
 30: 'win',
 31: 'mobile',
 32: 'n',
 33: 'phone',
 34: 'new',
 35: 'week',
 36: 'later',
 37: 'work',
 38: 'da',
 39: 'hi',
 40: 'ask',
 41: 'd',
 42: 'miss',
 43: 'ì',
 44: 'hope',
 45: 'night',
 46: 'claim',
 47: 's',
 48: 'thing',
 49: 'great',
 50: 'try',
 51: 'wait',
 52: 'oh',
 53: 'hey',
 54: 'leave',
 55: 'meet',
 56: 'dear',
 57: 'pls',
 58: 'happy',
 59: 'message',
 60: 'number',
 61: 'wat',
 62: 'friend',
 63: 'm',
 64: 'thank',
 65: 'feel',
 66: 'way',
 67: 'late',
 68: 'prize',
 69: 'right',
 70: 'find',
 71: 'pick',
 72: 'tomorrow',
 73: 'yes',
 74: 'yeah',
 75: '1',
 76: 'min',
 77: 'e',
 78: 'msg',
 79: 'c',
 80: '

### *Train test split*

*Tensorflow is incapable as of now to create a tensor dataset based on lists of different lengths, we will have to store all of our encoded texts into a single numpy array before creating the tensorflow dataset.
Not all our sequences are the same length, this is where the `tf.keras.preprocessing.sequence.pad_sequences` comes in handy, it will add zero padding at the beginning (`padding="pre"`) or at the end (`padding="post"`) of your sequences so they all have equal length.
Let's Pad the sequences*

In [ ]:
msg_pad = tf.keras.preprocessing.sequence.pad_sequences(dataset.msg_encoded, padding="post")

In [ ]:
# Train Test Split
xtrain, xval, ytrain, yval = train_test_split(msg_pad,dataset.type, test_size=0.3)

In [ ]:
train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
val = tf.data.Dataset.from_tensor_slices((xval, yval))

In [ ]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

In [ ]:
# Let's check one batch 
for msg, msg_type in train_batch.take(1):
  print(msg, msg_type)

tf.Tensor(
[[ 52   1 696 ...   0   0   0]
 [ 92 199  87 ...   0   0   0]
 [  1   1 508 ...   0   0   0]
 ...
 [ 64   1 199 ...   0   0   0]
 [172  32 654 ...   0   0   0]
 [  3   1   1 ...   0   0   0]], shape=(64, 74), dtype=int32) tf.Tensor(
[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0], shape=(64,), dtype=int64)


### *CLASSIFICATION MODELING* ###

### *SimpleRNN*

In [ ]:
vocab_size = tokenizer.num_words
model = tf.keras.Sequential([
                  # Couche d'Input Word Embedding           
                  tf.keras.layers.Embedding(vocab_size+1, 8, input_shape=[msg.shape[1],],name="embedding"),
                  # Gobal average pooling
                  tf.keras.layers.GlobalAveragePooling1D(),

                  # Couche Dense classique
                  tf.keras.layers.Dense(16, activation='relu'),

                  # Couche de sortie avec le nombre de neurones en sortie égale au nombre de classe avec fonction softmax
                  tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 74, 8)             8008      
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                144       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 8,169
Trainable params: 8,169
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer= tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy
              ()])

In [ ]:
history = model.fit(train_batch, 
                    epochs=50, 
                    validation_data=val_batch)
val_loss_simple_RNN, val_acc_simple_RNN = model.evaluate(xval, yval)
print('Val accuracy simple_RNN:', val_acc_simple_RNN)
print('Val loss simple_RNN:', val_loss_simple_RNN)

Epoch 1/50
61/61 [==============================] - 1s 6ms/step - loss: 0.6399 - binary_accuracy: 0.8305 - val_loss: 0.5792 - val_binary_accuracy: 0.8630
Epoch 2/50
61/61 [==============================] - 0s 4ms/step - loss: 0.5089 - binary_accuracy: 0.8672 - val_loss: 0.4424 - val_binary_accuracy: 0.8630
Epoch 3/50
61/61 [==============================] - 0s 3ms/step - loss: 0.3968 - binary_accuracy: 0.8672 - val_loss: 0.3747 - val_binary_accuracy: 0.8630
Epoch 4/50
61/61 [==============================] - 0s 4ms/step - loss: 0.3586 - binary_accuracy: 0.8672 - val_loss: 0.3597 - val_binary_accuracy: 0.8630
Epoch 5/50
61/61 [==============================] - 0s 4ms/step - loss: 0.3465 - binary_accuracy: 0.8672 - val_loss: 0.3487 - val_binary_accuracy: 0.8630
Epoch 6/50
61/61 [==============================] - 0s 3ms/step - loss: 0.3349 - binary_accuracy: 0.8672 - val_loss: 0.3362 - val_binary_accuracy: 0.8630
Epoch 7/50
61/61 [==============================] - 0s 4ms/step - loss: 0.32

In [ ]:
# Visualization of the training process on the loss function
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                          color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and validation on the loss function",
    xaxis_title="epochs",
    yaxis_title="Cross Entropy"    
)
fig.show()

*The results from the RNN model are good. The model continuously learns form the training examples, and starts overfitting after epoch number 20. This is a schoolbook example of model training right here*

*In addition to this, the MSE on the validation data is around 0.08*

In [ ]:
# Visualization of accuracy training 
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["binary_accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_binary_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="epochs",
    yaxis_title="Accuracy"    
)
fig.show()

### *GRU*

In [ ]:
vocab_size = tokenizer.num_words
model_gru = tf.keras.Sequential([
                  Embedding(vocab_size+1, 8, input_shape=[msg.shape[1],],name="embedding"),
                  GRU(units=64, return_sequences=True), # maintains the sequential nature
                  GRU(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid")
])

In [ ]:
model_gru.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 74, 8)             8008      
                                                                 
 gru_2 (GRU)                 (None, 74, 64)            14208     
                                                                 
 gru_3 (GRU)                 (None, 32)                9408      
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 32,297
Trainable params: 32,297
Non-trai

In [ ]:
model_gru.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Model training 
gru_history = model_gru.fit(train_batch,
                              epochs=20, 
                              validation_data=val_batch
)
val_loss_gru, val_acc_gru = model_gru.evaluate(xval, yval)
print("Val accuracy gru:", val_acc_gru)
print("Val loss gru:", val_loss_gru)

Epoch 1/20
61/61 [==============================] - 11s 112ms/step - loss: 0.4887 - accuracy: 0.8564 - val_loss: 0.3996 - val_accuracy: 0.8630
Epoch 2/20
61/61 [==============================] - 6s 98ms/step - loss: 0.3931 - accuracy: 0.8672 - val_loss: 0.4122 - val_accuracy: 0.8630
Epoch 3/20
61/61 [==============================] - 6s 99ms/step - loss: 0.3942 - accuracy: 0.8672 - val_loss: 0.4002 - val_accuracy: 0.8630
Epoch 4/20
61/61 [==============================] - 6s 98ms/step - loss: 0.3924 - accuracy: 0.8672 - val_loss: 0.4013 - val_accuracy: 0.8630
Epoch 5/20
61/61 [==============================] - 6s 99ms/step - loss: 0.3935 - accuracy: 0.8672 - val_loss: 0.4003 - val_accuracy: 0.8630
Epoch 6/20
61/61 [==============================] - 6s 99ms/step - loss: 0.3925 - accuracy: 0.8672 - val_loss: 0.4000 - val_accuracy: 0.8630
Epoch 7/20
61/61 [==============================] - 6s 99ms/step - loss: 0.3930 - accuracy: 0.8672 - val_loss: 0.3996 - val_accuracy: 0.8630
Epoch 8/20


In [ ]:
# Visualization of the training process on the loss function
fig = go.Figure(data=[
                      go.Scatter(
                          y=gru_history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                          color=color_chart[0]
                          )),
                      go.Scatter(
                          y=gru_history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and validation on the loss function",
    xaxis_title="epochs",
    yaxis_title="Cross Entropy"    
)
fig.show()

In [ ]:
# Visualization of accuracy training 
fig = go.Figure(data=[
                      go.Scatter(
                          y=gru_history.history["accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=gru_history.history["val_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="epochs",
    yaxis_title="Accuracy"    
)
fig.show()

### *LSTM*

In [ ]:
vocab_size = tokenizer.num_words
model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 8, input_shape=[msg.shape[1],],name="embedding"),
                  LSTM(units=64, return_sequences=True), # maintains the sequential nature
                  LSTM(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid")
])

In [ ]:
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 74, 8)             8008      
                                                                 
 lstm_4 (LSTM)               (None, 74, 64)            18688     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 8)                 136       
                                                                 
 dense_16 (Dense)            (None, 1)                 9         
                                                                 
Total params: 39,785
Trainable params: 39,785
Non-trai

In [ ]:
model_lstm.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Model training 
lstm_history = model_lstm.fit(train_batch,
                              epochs=20, 
                              validation_data=val_batch
)
val_loss_lstm, val_acc_lstm = model_lstm.evaluate(xval, yval)
print("Val accuracy lstm:", val_acc_lstm)
print("Val loss lstm:", val_loss_lstm)

Epoch 1/20
61/61 [==============================] - 11s 123ms/step - loss: 0.4388 - accuracy: 0.8672 - val_loss: 0.3994 - val_accuracy: 0.8630
Epoch 2/20
61/61 [==============================] - 7s 110ms/step - loss: 0.3928 - accuracy: 0.8672 - val_loss: 0.3994 - val_accuracy: 0.8630
Epoch 3/20
61/61 [==============================] - 7s 108ms/step - loss: 0.3924 - accuracy: 0.8672 - val_loss: 0.3995 - val_accuracy: 0.8630
Epoch 4/20
61/61 [==============================] - 7s 108ms/step - loss: 0.3928 - accuracy: 0.8672 - val_loss: 0.3996 - val_accuracy: 0.8630
Epoch 5/20
61/61 [==============================] - 7s 115ms/step - loss: 0.3927 - accuracy: 0.8672 - val_loss: 0.3998 - val_accuracy: 0.8630
Epoch 6/20
61/61 [==============================] - 7s 108ms/step - loss: 0.3932 - accuracy: 0.8672 - val_loss: 0.3994 - val_accuracy: 0.8630
Epoch 7/20
61/61 [==============================] - 7s 108ms/step - loss: 0.3920 - accuracy: 0.8672 - val_loss: 0.4006 - val_accuracy: 0.8630
Epoch

In [ ]:
# Visualization of the training process on the loss function
fig = go.Figure(data=[
                      go.Scatter(
                          y=lstm_history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                          color=color_chart[0]
                          )),
                      go.Scatter(
                          y=lstm_history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and validation on the loss function",
    xaxis_title="epochs",
    yaxis_title="Cross Entropy"    
)
fig.show()

In [ ]:
# Visualization of accuracy training 
fig = go.Figure(data=[
                      go.Scatter(
                          y=lstm_history.history["accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=lstm_history.history["val_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="epochs",
    yaxis_title="Accuracy"    
)
fig.show()

### *Model comparaison*

In [ ]:
# RNN model
print('Val accuracy simple_RNN:', val_acc_simple_RNN)
print('Val loss simple_RNN:', val_loss_simple_RNN)

Val accuracy simple_RNN: 0.9832535982131958
Val loss simple_RNN: 0.08369994163513184


In [ ]:
# GRU model 
print("Val accuracy gru:", val_acc_gru)
print("Val loss gru:", val_loss_gru)

Val accuracy gru: 0.8630383014678955
Val loss gru: 0.3996928334236145


In [ ]:
# LSTM model
print("Val accuracy gru:", val_acc_lstm)
print("Val loss gru:", val_loss_lstm)

Val accuracy gru: 0.9246411323547363
Val loss gru: 0.2627447247505188


*We are better able to detect spam with the simple RNN model rather than GRU model and LSTM model.*
